In [45]:
!pip install rouge_score -q

In [46]:
!pip install --upgrade transformers -q

In [47]:
#import thư viện
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    #AdamW,
    get_linear_schedule_with_warmup,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from torch.optim import AdamW
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset as HFDataset
import json
import os
from tqdm import tqdm
import logging
from typing import Dict, List, Optional, Tuple
import wandb
from rouge_score import rouge_scorer
import warnings

warnings.filterwarnings('ignore')


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [49]:
# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [50]:
def setup_config():
    """Tập trung toàn bộ cấu hình vào một nơi."""
    config = {
        "task_type": "summarization",
        "csv_file_path": "/kaggle/input/all-data/data_fireant_0407_final.xlsx - Sheet1.tsv",
        "num_epochs": 3,
        "batch_size": 2,
        "learning_rate": 3e-5,
        "model_name": "VietAI/vit5-base",
        "output_dir": "./vit5_summarization_finetuned",
        "max_input_length": 1024,
        "max_target_length": 256,
        "validation_split_size": 0.15,
        "random_state": 42,
        # New parameters to reduce repetition
        "repetition_penalty": 1.2,
        "no_repeat_ngram_size": 3,
        "length_penalty": 1.0,
        "num_beams": 4,
        "early_stopping": True,
        "do_sample": False,  # Set to True for more diverse outputs
        "temperature": 1.0,
        "top_k": 50,
        "top_p": 0.95
    }
    logger.info(f"📊 Cấu hình đã được thiết lập: {config}")
    return config

In [51]:
def load_model_and_tokenizer(model_name: str):
    """Tải pre-trained model và tokenizer."""
    logger.info(f"🔄 Đang tải model và tokenizer: '{model_name}'...")
    
    tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    logger.info(f"✅ Tải thành công model '{model_name}' trên device '{device}'")
    return model, tokenizer, device

In [ ]:
def load_data_from_tsv(file_path: str) -> list:
    """
    Tải dữ liệu từ file TSV và chuyển đổi thành định dạng cần thiết.

    Args:
        file_path: Đường dẫn đến file TSV.

    Returns:
        Một list các dictionary, mỗi dict chứa 'input' và 'target'.
    """

    try:
        logger.info(f"Reading data from file: {file_path}")
        df = pd.read_csv(file_path, sep='\t')

        # Check for required columns
        if 'content' not in df.columns or 'summary' not in df.columns:
            raise ValueError("Error: TSV file must contain 'content' and 'summary' columns.")

        # Rename columns to match format {'input': ..., 'target': ...}
        df = df.rename(columns={'content': 'input', 'summary': 'target'})
        
        # Remove rows with empty values in input or target columns
        df.dropna(subset=['input', 'target'], inplace=True)
        
        # Clean the data - remove extra whitespace and ensure proper format
        df['input'] = df['input'].astype(str).str.strip()
        df['target'] = df['target'].astype(str).str.strip()
        
        # Filter out very short summaries that might cause repetition issues
        df = df[df['target'].str.len() > 10]
        df = df[df['input'].str.len() > 50]

        logger.info(f"✅ Successfully read {len(df)} samples from TSV file.")
        return df.to_dict('records')

    except FileNotFoundError:
        logger.error(f"❌ File not found at path: {file_path}")
        raise
    except Exception as e:
        logger.error(f"❌ Error reading TSV file: {e}")
        raise
    


In [ ]:
def prepare_datasets(config: dict, tokenizer: T5Tokenizer):
    """Load, split and preprocess tokenize data."""
    logger.info("🛠️  Starting data preparation...")

    # Load and split data
    all_data = load_data_from_tsv(config['csv_file_path'])
    train_data_list, eval_data_list = train_test_split(
        all_data,
        test_size=config['validation_split_size'],
        random_state=config['random_state']
    )
    train_dataset = HFDataset.from_pandas(pd.DataFrame(train_data_list))
    eval_dataset = HFDataset.from_pandas(pd.DataFrame(eval_data_list))
    logger.info(f"✅ Data split: {len(train_dataset)} training samples, {len(eval_dataset)} validation samples.")

    # Preprocessing function
    task_prefix = "summarize: "
    def preprocess_function(examples):
        inputs = [task_prefix + doc for doc in examples["input"]]
        model_inputs = tokenizer(
            inputs, 
            max_length=config['max_input_length'], 
            truncation=True, 
            padding="max_length"
        )
        
        # Process targets with proper handling
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                examples["target"], 
                max_length=config['max_target_length'], 
                truncation=True, 
                padding="max_length"
            )
        
        # Replace padding token id with -100 for loss calculation
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] 
            for label in labels["input_ids"]
        ]
        
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Tokenize
    tokenized_train_dataset = train_dataset.map(
        preprocess_function, 
        batched=True, 
        remove_columns=train_dataset.column_names
    )
    tokenized_eval_dataset = eval_dataset.map(
        preprocess_function, 
        batched=True, 
        remove_columns=eval_dataset.column_names
    )
    logger.info("✅ Data tokenized successfully.")
    
    return tokenized_train_dataset, tokenized_eval_dataset
    

In [54]:
def compute_metrics(eval_pred):
    """Compute ROUGE metrics for evaluation."""
    predictions, labels = eval_pred
    
    # Decode predictions and labels
    tokenizer = T5Tokenizer.from_pretrained("VietAI/vit5-base", legacy=False)
    
    # Replace -100 with pad token id for decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    
    for pred, label in zip(decoded_preds, decoded_labels):
        scores = scorer.score(label, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    
    return {
        'rouge1': np.mean(rouge1_scores),
        'rouge2': np.mean(rouge2_scores),
        'rougeL': np.mean(rougeL_scores)
    }

In [ ]:
def setup_trainer(model, tokenizer, config, train_dataset, eval_dataset):
    """Configure and initialize Hugging Face Trainer with improved settings."""
    logger.info("⚙️  Configuring Hugging Face Trainer...")
    
    training_args = TrainingArguments(
        output_dir=config['output_dir'],
        num_train_epochs=config['num_epochs'],
        per_device_train_batch_size=config['batch_size'],
        per_device_eval_batch_size=config['batch_size'],
        learning_rate=config['learning_rate'],
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir=f"{config['output_dir']}/logs",
        logging_steps=50,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss", 
        greater_is_better=False,
        fp16=torch.cuda.is_available(),
        report_to="none",
        # Additional parameters to improve training stability
        gradient_accumulation_steps=1,
        dataloader_pin_memory=False,
        remove_unused_columns=True,
    )

    data_collator = DataCollatorForSeq2Seq(
        tokenizer, 
        model=model, 
        padding=True,
        return_tensors="pt"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=None,  # We'll compute metrics separately
    )
    
    return trainer



In [56]:
def generate_summary_with_improved_settings(model, tokenizer, input_text, config, device):
    """Generate summary with improved settings to reduce repetition."""
    
    # Prepare input
    input_text = "summarize: " + input_text
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=config['max_input_length'],
        truncation=True,
        padding=True
    ).input_ids.to(device)
    
    # Generate with improved parameters
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=config['max_target_length'],
            min_length=30,  # Ensure minimum length
            num_beams=config['num_beams'],
            repetition_penalty=config['repetition_penalty'],
            no_repeat_ngram_size=config['no_repeat_ngram_size'],
            length_penalty=config['length_penalty'],
            early_stopping=config['early_stopping'],
            do_sample=config['do_sample'],
            temperature=config['temperature'] if config['do_sample'] else None,
            top_k=config['top_k'] if config['do_sample'] else None,
            top_p=config['top_p'] if config['do_sample'] else None,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode the generated summary
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_summary

In [ ]:
def run_training_pipeline():
    """
    Main function to orchestrate the entire fine-tuning process.
    """
    logger.info("🚀 STARTING VIT5 FINE-TUNING PIPELINE")
    print("=" * 60)

    config = setup_config()
    model, tokenizer, device = load_model_and_tokenizer(config['model_name'])
    train_ds, eval_ds = prepare_datasets(config, tokenizer)
    trainer = setup_trainer(model, tokenizer, config, train_ds, eval_ds)
    
    logger.info("🎯 Starting fine-tuning process...")
    trainer.train()
    logger.info("✅ Fine-tuning completed!")
    
    final_model_path = f"{config['output_dir']}/final_model"
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    logger.info(f"💾 Best model saved at: {final_model_path}")

    logger.info("📈 Evaluating model on validation set...")
    eval_results = trainer.evaluate()
    logger.info(f"   - Eval Loss: {eval_results.get('eval_loss', 'N/A')}")
    logger.info(f"   - Eval Runtime: {eval_results.get('eval_runtime', 'N/A')}s")

    logger.info("\n🧪 Testing fine-tuned model:")
    test_input = """Địa ốc First Real dự kiến phát hành hơn 6,4 triệu cổ phiếu thưởng cho cổ đông nhằm tăng vốn điều lệ, ngày đăng ký cuối cùng để phân bổ quyền là 30/7/2025.
Công ty Cổ phần Địa ốc First Real (MCK: FIR, sàn HoSE) vừa có văn bản thông báo về phát hành cổ phiếu để tăng vốn cổ phần từ nguồn vốn chủ sở hữu.
Theo đó, Địa ốc First Real dự kiến phát hành hơn 6,4 triệu cổ phiếu cho cổ đông hiện hữu với tỷ lệ thực hiện quyền 10:1, tức cổ đông sở hữu 1 cổ phiếu được hưởng 1 quyền, cứ 10 quyền sẽ được nhận 1 cổ phiếu mới. Ngày đăng ký cuối cùng để phân bổ quyền là 30/7/2025.
Tổng giá trị phát hành tính theo mệnh giá là hơn 64,2 tỷ đồng. Nguồn vốn thực hiện được lấy từ nguồn thặng dư vốn cổ phần của công ty theo báo cáo tài chính năm 2024 đã kiểm toán.
Ảnh minh họa
Nếu đợt phát hành thành công, số lượng cổ phiếu đã phát hành của Địa ốc First Real sẽ tăng từ hơn 64,2 triệu cổ phiếu lên gần 70,7 triệu cổ phiếu, tương đương vốn điều lệ tăng từ gần 642,5 tỷ đồng lên gần 706,7 tỷ đồng.
Được biết, phương án phát hành cổ phiếu này đã được cổ đông của Địa ốc First Real thông qua tại Đại hội đồng cổ đông (ĐHĐCĐ) thường niên 2025 được tổ chức ngày 21/3/2025."""
        
    loaded_tokenizer = T5Tokenizer.from_pretrained(final_model_path, legacy=False)
    loaded_model = T5ForConditionalGeneration.from_pretrained(final_model_path).to(device)

    # Test with improved generation settings
    prediction = generate_summary_with_improved_settings(
        loaded_model, loaded_tokenizer, test_input, config, device
    )

    print("-" * 60)
    print(f"📌 Input: {test_input[:200]}...")
    print(f"💡 Improved Output: {prediction}")
    print("-" * 60)

if __name__ == "__main__":
    try:
        run_training_pipeline()
    except Exception as e:
        logger.error(f"❌ Critical error during execution: {e}")
        import traceback
        traceback.print_exc()


    

Map:   0%|          | 0/4684 [00:00<?, ? examples/s]

Map:   0%|          | 0/827 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.234600,1.127015
2,1.032000,1.097187
3,0.926700,1.102287


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


------------------------------------------------------------
📌 Input: Địa ốc First Real dự kiến phát hành hơn 6,4 triệu cổ phiếu thưởng cho cổ đông nhằm tăng vốn điều lệ, ngày đăng ký cuối cùng để phân bổ quyền là 30/7/2025.
Công ty Cổ phần Địa ốc First Real (MCK: FIR, ...
💡 Improved Output: Địa ốc First Real dự kiến phát hành hơn 6,4 triệu cổ phiếu thưởng cho cổ đông để tăng vốn điều lệ, ngày đăng ký cuối cùng là 30/7/2025. Tổng giá trị phát hành là hơn 64,2 tỷ đồng, được lấy từ thặng dư vốn cổ phần của công ty. Phương án này đã được cổ đông thông qua tại Đại hội đồng cổ đông thường niên năm 2015. Dự kiến số lượng cổ phiếu đã phát hành sẽ tăng từ gần 642,5 tỷ đồng lên gần 706,7 tỷ đồng.
------------------------------------------------------------


In [ ]:
!zip -r vit5_summarization_finetuned.zip /kaggle/working/vit5_summarization_finetuned